In [1]:
#setup
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import gensim

sns.set_theme()
plt.rcParams["figure.figsize"] = (10,8)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from main import nlp_pipeline_result

In [3]:
data_non_ner = nlp_pipeline_result(
    disable_ner=False,
    parameters={
        "remove_hashtags": True,
        "replace_politics": True,
        "replace_parties": True,
    }
)
data_non_ner['n_lemmas'] = data_non_ner['Lemmas'].str.split().str.len()

#data_non_ner.to_csv('data/data.csv',index = False)



SyntaxError: positional argument follows keyword argument (<ipython-input-3-ad5961dd94ed>, line 7)

In [ ]:
data = pd.read_csv('data/data.csv')

In [ ]:
for i in range(0,50,10):
 display(data[i:i+10])

In [ ]:
plt.figure()
data['n_lemmas'].value_counts().sort_index().plot(kind="bar",width=1,label='number of lemmas')
plt.axvline(x = data['n_lemmas'].mean(),label='mean',color = 'black',linestyle = '--')
plt.axvline(x = data['n_lemmas'].mode()[0],label='mode',color = 'red',linestyle = '--')
plt.axvline(x = data['n_lemmas'].median(),label='median',color = 'green',linestyle = '--')
plt.legend()
plt.title('number of lemmas in tweet distribution')
plt.xlabel('number of lemmas')
plt.ylabel('number of tweets')
plt.show()

In [ ]:
data = data[data['n_lemmas']>=3]
print('Number of tweets in DataFrame after removing short tweets:', len(data))

In [ ]:
tweets_corpus = data["Lemmas"].tolist()
tweets_corpus = list(set(tweets_corpus))
tweets_corpus = [el.split() for el in tweets_corpus]

# display(tweets_corpus)

In [ ]:
gensim.corpora.dictionary

In [ ]:
token_dic = gensim.corpora.Dictionary(tweets_corpus)

# Filter token dictionary

no_below = 5  # Minimum number of documents to keep a term in the dictionary
no_above = .4  # Maximum proportion of documents in which a term can appear to be kept in the dictionary


# BOW: Transform list of tokens into list of tuples (token id, token # of occurrences)

tweets_corpus_bow = [token_dic.doc2bow(doc) for doc in tweets_corpus]

tweets_corpus_bow

In [ ]:
from gensim.models.wrappers import LdaMallet
import os

os.environ['MALLET_HOME'] = r'C:/mallet'
mallet_path = 'C:/mallet/bin/mallet'

ldamallet = LdaMallet(mallet_path, corpus=tweets_corpus_bow, num_topics=20, id2word=token_dic, alpha=5, iterations=100)

In [ ]:
topn = 25

fig, axes = plt.subplots(4, 5, figsize=(16, 20), sharex=True)

for i in range(5):

    # Build a dataframe with columns 'token' and 'weight' for topic i
    df = pd.DataFrame(ldamallet.show_topic(i, topn=topn), columns=['token','weight'])
    sns.barplot(x='weight', y='token', data=df, color='c', orient='h', ax=axes[0][i])
    axes[0][i].set_title('Topic ' + str(i))
    # Build a dataframe with columns 'token' and 'weight' for topic i + 5
    df = pd.DataFrame(ldamallet.show_topic(i+5, topn=topn), columns=['token','weight'])
    sns.barplot(x='weight', y='token', data=df, color='c', orient='h', ax=axes[1][i])
    axes[1][i].set_title('Topic ' + str(i+5))
    # Build a dataframe with columns 'token' and 'weight' for topic i + 10
    df = pd.DataFrame(ldamallet.show_topic(i+10, topn=topn), columns=['token','weight'])
    sns.barplot(x='weight', y='token', data=df, color='c', orient='h', ax=axes[2][i])
    axes[2][i].set_title('Topic ' + str(i+10))
    # Build a dataframe with columns 'token' and 'weight' for topic i + 15
    df = pd.DataFrame(ldamallet.show_topic(i + 15, topn=topn), columns=['token','weight'])
    sns.barplot(x='weight', y='token', data=df, color='c', orient='h', ax=axes[3][i])
    axes[3][i].set_title('Topic ' + str(i + 15))
    
plt.show()

In [ ]:
train_vecs = []

for i in range(len(data)):
    top_topics = ldamallet.get_document_topics(train_corpus[i], minimum_probability=0.0)
    lda_topics_vector = [top_topics[i][1] for i in range(number_of_topics)]
    lda_topics_vector.extend([my_dataframe.iloc[i].favs])  # count of favs
    lda_topics_vector.extend([len(my_dataframe.iloc[i].retweets)])  # count of retweets
    train_vecs.append(topic_vec)
